In [1]:
## All the external libraries
import re
import csv
from getpass import getpass
from time import sleep
import random

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
#from msedge.selenium_tools import Edge, EdgeOptions
from selenium.webdriver import Chrome

In [2]:
## All the internal libraries

In [3]:
####### All the functions #######

## Get random time offset
def get_rand_offset(t=4):
    r = random.randint(0, t)
    r += random.random()
    return r

#### Get the browser
## Return <class 'selenium.webdriver.chrome.webdriver.WebDriver'>
## Opens an empty browser
def get_driver():
    if driver == None:
        return webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    else:
        return driver

## Go to address
def visit_site(site="https://www.google.com"):
    driver.get(site)
    
## Add tweets to file
def write_to_file(fname, data):
    pass

## Get n tweets
def get_ntweets(num_tweets=100):
    data = []
    tweet_ids = set()
    last_position = driver.execute_script("return window.pageYOffset;")
    scrolling = True

    while scrolling:
        page_cards = []
        while len(page_cards) < 1:
            page_cards = driver.find_elements_by_xpath('//article[@data-testid="tweet"]')

        min_tweets = -1*min(len(page_cards), 15)

        for card in page_cards[min_tweets:]:
            tweet = get_tweet_data(card)
            if tweet:
                tweet_id = ''.join(tweet)
                if tweet_id not in tweet_ids:
                    tweet_ids.add(tweet_id)
                    if len(data) == num_tweets:
                        scrolling = False
                        break
                    data.append(tweet)

        scroll_attempt = 0
        while True:
            # check scroll position
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            sleep(1+get_rand_offset(2))
            curr_position = driver.execute_script("return window.pageYOffset;")
            if last_position == curr_position:
                scroll_attempt += 1

                # end of scroll region
                if scroll_attempt >= 3:
                    scrolling = False
                    break
                else:
                    sleep(1+get_rand_offset(2)) # attempt another scroll
            else:
                last_position = curr_position
                break
    return data

## Get specific date tweets
def get_specific_date_tweets(search_term, date, num_tweets=100):
    url_to_visit = 'https://twitter.com/search?f=live&q=(%23Covid19)%20lang%3Aen
    '%20until%3A2020-05-02%20since%3A2020-05-01&src=typed_query'
    visit_site(site=url_to_visit)
    tweets = get_ntweets(num_tweets=num_tweets)
    write_to_file(fname=fname, data=tweets)
    pass

    

## Description
## Takes
## Returns
def get_tweet_data(card):
    """Extract data from tweet card"""
    username = card.find_element_by_xpath('.//span').text
    try:
        handle = card.find_element_by_xpath('.//span[contains(text(), "@")]').text
    except NoSuchElementException:
        return
    
    try:
        postdate = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    
    comment = card.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
    responding = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
    text = comment + responding
    reply_cnt = card.find_element_by_xpath('.//div[@data-testid="reply"]').text
    retweet_cnt = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    like_cnt = card.find_element_by_xpath('.//div[@data-testid="like"]').text
    
    # get a string of all emojis contained in the tweet
    """Emojis are stored as images... so I convert the filename, which is stored as unicode, into 
    the emoji character."""
    emoji_tags = card.find_elements_by_xpath('.//img[contains(@src, "emoji")]')
    emoji_list = []
    for tag in emoji_tags:
        filename = tag.get_attribute('src')
        try:
            emoji = chr(int(re.search(r'svg\/([a-z0-9]+)\.svg', filename).group(1), base=16))
        except AttributeError:
            continue
        if emoji:
            emoji_list.append(emoji)
    emojis = ' '.join(emoji_list)
    
    tweet = (username, handle, postdate, text, emojis, reply_cnt, retweet_cnt, like_cnt)
    return tweet   

In [4]:
## Initialization: To be run only once at the begining
driver = None

In [5]:
## Get the browser
driver = get_driver()



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
There is no [mac64] chromedriver for browser  in cache
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_mac64.zip
Driver has been saved in cache [/Users/wasim/.wdm/drivers/chromedriver/mac64/99.0.4844.51]


In [7]:
## Webdriver support following
print(f'dir of webdriver: {dir(driver)}')

dir of webdriver: ['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_file_detector', '_get_cdp_details', '_is_remote', '_mobile', '_shadowroot_cls', '_switch_to', '_unwrap_value', '_web_element_cls', '_wrap_value', 'add_cookie', 'application_cache', 'back', 'bidi_connection', 'capabilities', 'caps', 'close', 'command_executor', 'create_options', 'create_web_element', 'current_url', 'current_window_handle', 'delete_all_cookies', 'delete_cookie', 'delete_network_conditions', 'desired_capabilities', 'error_handler', 'execute', 'execute_async_script', 'execute_cdp_cmd', 'execute_script', 'file_detector', 'file_detector_context', 'find_elem

In [8]:
## Visit Google
visit_site()

In [9]:
## Visit twitter
visit_site(site='https://twitter.com/')

In [10]:
twitter_cookies = driver.get_cookies()

In [11]:
print(twitter_cookies)

[{'domain': '.twitter.com', 'expiry': 1646798151, 'httpOnly': True, 'name': 'att', 'path': '/', 'sameSite': 'None', 'secure': True, 'value': '1-yNFMLDh5Su1EvMbeHSgjcBmxDu9TlUObLZFHxTvw'}, {'domain': '.twitter.com', 'expiry': 1646722514, 'httpOnly': False, 'name': 'gt', 'path': '/', 'secure': True, 'value': '1501043840580349956'}, {'domain': '.twitter.com', 'expiry': 1709783718, 'httpOnly': False, 'name': '_ga', 'path': '/', 'secure': False, 'value': 'GA1.2.1984098986.1646711718'}, {'domain': '.twitter.com', 'expiry': 1693972551, 'httpOnly': True, 'name': 'kdt', 'path': '/', 'secure': True, 'value': 'uhW8ppftr57SEcdf9AWo5E3AeOBNzMtGkWkNJb3v'}, {'domain': '.twitter.com', 'expiry': 1646798118, 'httpOnly': False, 'name': '_gid', 'path': '/', 'secure': False, 'value': 'GA1.2.1338411382.1646711718'}, {'domain': '.twitter.com', 'expiry': 1709783712, 'httpOnly': False, 'name': 'personalization_id', 'path': '/', 'sameSite': 'None', 'secure': True, 'value': '"v1_wtb2MCio99tVklwUnuJgWw=="'}, {'do

In [12]:
## Deletes the current cookies
driver.delete_all_cookies()

In [15]:
## Restore the cookie
for cookie in twitter_cookies:
    #print(f'cookie:{cookie}')
    driver.add_cookie(cookie_dict=cookie)

In [17]:
## Refreshes the page
driver.refresh()

In [20]:
## Search
visit_site(site='https://twitter.com/search?f=live&q=(%23Covid19)%20lang%3Aen%20until%3A2020-05-02%20since%3A2020-05-01&src=typed_query')

In [21]:
# get all tweets on the page
data = get_ntweets(num_tweets=5)

/var/folders/95/zkn8qt652zv_1vjspb00z53h0000gn/T/ipykernel_32845/1248779822.py:36: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  page_cards = driver.find_elements_by_xpath('//article[@data-testid="tweet"]')
/Users/wasim/opt/miniconda3/envs/ml_dl_39/lib/python3.9/site-packages/selenium/webdriver/remote/webelement.py:393: UserWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  warnings.warn("find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead")
/Users/wasim/opt/miniconda3/envs/ml_dl_39/lib/python3.9/site-packages/selenium/webdriver/remote/webelement.py:426: UserWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  warnings.warn("find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead")


In [22]:
print(data)

[('Ed Ward MD', '@EdWardMDBlog', '2020-05-01T23:59:57.000Z', 'Ed Ward MD\n@EdWardMDBlog\n·\nMay 1, 2020Ed Ward\nApril 15 at 3:26 PM · \n#GileadSciences #remdesivir #ICYMI #COVID2019 #COVID19 #CoronaVirus #SARSCov2 #EffectiveTreatment 1 #Oxygen 2 Remdesivir #AntiViral 3 If infection #Azithromycin add 4 #Roche #Actemra if #AntiCytokineStorm #DrEd #EdWardMD #TwitterCensors "Ed Ward MD"\n1\n4\n1', '', '1', '4', '1'), ('Archdiocese of STL', '@archstl', '2020-05-01T23:59:54.000Z', 'Archdiocese of STL\n@archstl\n·\nMay 1, 2020Tune in at http://cathedralstl.org now and support \n@cc_stl\n ‘s #COVID19 relief efforts!\nFOX2now\n@FOX2now\n · May 1, 2020\nCathedral Basilica of St. Louis to host online concert fundraiser https://fox2now.com/news/coronavirus/cathedral-basilica-of-st-louis-to-host-online-concert-fundraiser/?taid=5eacb238d9c55b000110d67a&utm_campaign=trueAnthem%3A+New+Content+%28Feed%29&utm_medium=trueAnthem&utm_source=twitter…\n2', '', '', '', '2'), ('Robert Hanka', '@HankaRobert', '

In [23]:
len(data)

5